# NBA Analysis

## NBA Data Gathering

In this section, I will display how the data was gathered. I will primarily focus on features, allowing me to evaluate the player's efficiency. To do so, the following metrics will be focused on:

- Effective field goal percentage (eFG%)
- True shooting percentage (TS%)
- Player efficiency rating (PER)
- FG => 2P & 3P

(Tiodorovic, 2020), (STACK NJ/NY, 2021), (basketball stats, 2021)

In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
from bs4 import BeautifulSoup
from bs4 import Comment
import requests

The following section will show all the tables I got from the website. During the analysis, I will decide which tables and features are essential.

In [2]:
pd.set_option('display.max_columns',100)
total_stats=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_totals.html')[0]
total_stats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,55,12,1140,196,404,.485,29,108,.269,167,296,.564,.521,87,124,.702,100,228,328,50,31,30,59,102,508
1,2,Steven Adams,C,29,MEM,42,42,1133,157,263,.597,0,1,.000,157,262,.599,.597,47,129,.364,214,271,485,97,36,46,79,98,361
2,3,Bam Adebayo,C,25,MIA,75,75,2598,602,1114,.540,1,12,.083,601,1102,.545,.541,324,402,.806,184,504,688,240,88,61,187,208,1529
3,4,Ochai Agbaji,SG,22,UTA,59,22,1209,165,386,.427,81,228,.355,84,158,.532,.532,56,69,.812,43,78,121,67,16,15,41,99,467
4,5,Santi Aldama,PF,22,MEM,77,20,1682,247,525,.470,94,266,.353,153,259,.591,.560,108,144,.750,85,286,371,97,45,48,60,143,696


In [2]:
avg_stats=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_per_minute.html')[0]
avg_stats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,55,12,1140,6.2,12.8,...,.702,3.2,7.2,10.4,1.6,1.0,0.9,1.9,3.2,16.0
1,2,Steven Adams,C,29,MEM,42,42,1133,5.0,8.4,...,.364,6.8,8.6,15.4,3.1,1.1,1.5,2.5,3.1,11.5
2,3,Bam Adebayo,C,25,MIA,75,75,2598,8.3,15.4,...,.806,2.5,7.0,9.5,3.3,1.2,0.8,2.6,2.9,21.2
3,4,Ochai Agbaji,SG,22,UTA,59,22,1209,4.9,11.5,...,.812,1.3,2.3,3.6,2.0,0.5,0.4,1.2,2.9,13.9
4,5,Santi Aldama,PF,22,MEM,77,20,1682,5.3,11.2,...,.750,1.8,6.1,7.9,2.1,1.0,1.0,1.3,3.1,14.9


In [4]:
hundert_poss=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_per_poss.html')[0]
hundert_poss.drop(['Unnamed: 29'], axis=1, inplace=True)
hundert_poss.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,1,Precious Achiuwa,C,23,TOR,55,12,1140,8.5,17.5,.485,1.3,4.7,.269,7.2,12.8,.564,3.8,5.4,.702,4.3,9.9,14.2,2.2,1.3,1.3,2.6,4.4,22.0,112,113
1,2,Steven Adams,C,29,MEM,42,42,1133,6.6,11.0,.597,0.0,0.0,.000,6.6,11.0,.599,2.0,5.4,.364,9.0,11.4,20.3,4.1,1.5,1.9,3.3,4.1,15.1,118,108
2,3,Bam Adebayo,C,25,MIA,75,75,2598,11.6,21.4,.540,0.0,0.2,.083,11.5,21.2,.545,6.2,7.7,.806,3.5,9.7,13.2,4.6,1.7,1.2,3.6,4.0,29.3,115,111
3,4,Ochai Agbaji,SG,22,UTA,59,22,1209,6.5,15.3,.427,3.2,9.0,.355,3.3,6.2,.532,2.2,2.7,.812,1.7,3.1,4.8,2.6,0.6,0.6,1.6,3.9,18.5,115,121
4,5,Santi Aldama,PF,22,MEM,77,20,1682,7.0,14.8,.470,2.7,7.5,.353,4.3,7.3,.591,3.0,4.1,.750,2.4,8.1,10.5,2.7,1.3,1.4,1.7,4.0,19.7,120,111


In [5]:
advanced=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_advanced.html')[0]
advanced.drop(['Unnamed: 19','Unnamed: 24'], axis=1, inplace=True)
advanced.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Precious Achiuwa,C,23,TOR,55,1140,15.2,.554,.267,.307,9.3,24.4,16.3,6.3,1.3,2.6,11.4,19.4,0.8,1.4,2.2,.093,-1.4,-0.8,-2.3,-0.1
1,2,Steven Adams,C,29,MEM,42,1133,17.5,.564,.004,.490,20.1,25.3,22.7,11.2,1.5,3.7,19.8,14.6,1.3,2.1,3.4,.144,-0.3,0.9,0.6,0.7
2,3,Bam Adebayo,C,25,MIA,75,2598,20.1,.592,.011,.361,8.0,23.6,15.5,15.9,1.7,2.4,12.7,25.2,3.6,3.8,7.4,.137,0.8,0.8,1.5,2.3
3,4,Ochai Agbaji,SG,22,UTA,59,1209,9.5,.561,.591,.179,3.9,6.9,5.4,7.5,0.6,1.0,9.0,15.8,0.9,0.4,1.3,.053,-1.7,-1.4,-3.0,-0.3
4,5,Santi Aldama,PF,22,MEM,77,1682,13.9,.591,.507,.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,2.1,2.4,4.6,.130,-0.3,0.8,0.5,1.1


In [6]:
play_by_play=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_play-by-play.html')[0]
play_by_play=play_by_play.droplevel(0,axis=1)
play_by_play.head()

,Rk,Player,Pos,Age,Tm,G,MP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
0,1,Precious Achiuwa,C,23,TOR,55,1140,NaN,NaN,NaN,17%,83%,-1.5,-3.8,8,21,53,19,58,7,128,15,31
1,2,Steven Adams,C,29,MEM,42,1133,NaN,NaN,NaN,NaN,100%,9.5,8.3,40,10,40,23,51,7,223,15,14
2,3,Bam Adebayo,C,25,MIA,75,2598,NaN,NaN,NaN,NaN,100%,1.7,5.4,60,46,83,50,194,10,587,37,49
3,4,Ochai Agbaji,SG,22,UTA,59,1209,38%,40%,20%,3%,NaN,-0.5,0.9,15,8,65,9,32,3,159,4,20
4,5,Santi Aldama,PF,22,MEM,77,1682,NaN,NaN,2%,69%,29%,2.2,-2.5,19,20,93,11,48,11,231,12,20


In [7]:
shooting=pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_shooting.html')[0]
shooting=shooting.droplevel(0,axis=1)
shooting.drop(['Unnamed: 9_level_1','Unnamed: 16_level_1','Unnamed: 23_level_1','Unnamed: 26_level_1',
              'Unnamed: 29_level_1','Unnamed: 32_level_1'],axis=1,inplace=True)
shooting.head()

,Rk,Player,Pos,Age,Tm,G,MP,FG%,Dist.,2P,0-3,3-10,10-16,16-3P,3P,2P,0-3,3-10,10-16,16-3P,3P,2P,3P,%FGA,#,%3PA,3P%,Att.,#
0,1,Precious Achiuwa,C,23,TOR,55,1140,.485,9.7,.733,.381,.262,.069,.012,.267,.564,.734,.406,.286,.200,.269,.623,.966,.141,49,.444,.250,1,0
1,2,Steven Adams,C,29,MEM,42,1133,.597,2.6,.996,.696,.281,.019,.000,.004,.599,.645,.500,.400,NaN,.000,.503,NaN,.163,41,1.000,.000,0,0
2,3,Bam Adebayo,C,25,MIA,75,2598,.540,7.3,.989,.295,.379,.260,.047,.011,.545,.714,.479,.497,.288,.083,.594,1.000,.136,139,.000,NaN,1,0
3,4,Ochai Agbaji,SG,22,UTA,59,1209,.427,16.9,.409,.150,.210,.026,.023,.591,.532,.621,.494,.400,.444,.355,.762,.988,.054,17,.412,.457,0,0
4,5,Santi Aldama,PF,22,MEM,77,1682,.470,14.6,.493,.280,.166,.030,.017,.507,.591,.707,.471,.250,.444,.353,.680,.968,.118,54,.391,.365,1,0


In [8]:
#here, I have created the final data frame, which consists of total_stats, advanced stats, and play-by-play stats
df_final=pd.merge(total_stats,advanced, how='left', on=['Player','Pos','Age','Tm','G'])
df_final.drop(['Rk_x','Rk_y','MP_y'], axis=1, inplace=True)
df_final.rename(columns={'MP_x':'MP'},inplace=True)
df_final=pd.merge(df_final, play_by_play, how='left', on=['Player','Pos','Age','Tm','G'])
df_final.drop(['Rk','MP_y'], axis=1, inplace=True)
df_final.rename(columns={'MP_x':'MP'},inplace=True)
df_final.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
0,Precious Achiuwa,C,23,TOR,55,12,1140,196,404,.485,29,108,.269,167,296,.564,.521,87,124,.702,100,228,328,50,31,30,59,102,508,15.2,.554,.267,.307,9.3,24.4,16.3,6.3,1.3,2.6,11.4,19.4,0.8,1.4,2.2,.093,-1.4,-0.8,-2.3,-0.1,NaN,NaN,NaN,17%,83%,-1.5,-3.8,8,21,53,19,58,7,128,15,31
1,Steven Adams,C,29,MEM,42,42,1133,157,263,.597,0,1,.000,157,262,.599,.597,47,129,.364,214,271,485,97,36,46,79,98,361,17.5,.564,.004,.490,20.1,25.3,22.7,11.2,1.5,3.7,19.8,14.6,1.3,2.1,3.4,.144,-0.3,0.9,0.6,0.7,NaN,NaN,NaN,NaN,100%,9.5,8.3,40,10,40,23,51,7,223,15,14
2,Bam Adebayo,C,25,MIA,75,75,2598,602,1114,.540,1,12,.083,601,1102,.545,.541,324,402,.806,184,504,688,240,88,61,187,208,1529,20.1,.592,.011,.361,8.0,23.6,15.5,15.9,1.7,2.4,12.7,25.2,3.6,3.8,7.4,.137,0.8,0.8,1.5,2.3,NaN,NaN,NaN,NaN,100%,1.7,5.4,60,46,83,50,194,10,587,37,49
3,Ochai Agbaji,SG,22,UTA,59,22,1209,165,386,.427,81,228,.355,84,158,.532,.532,56,69,.812,43,78,121,67,16,15,41,99,467,9.5,.561,.591,.179,3.9,6.9,5.4,7.5,0.6,1.0,9.0,15.8,0.9,0.4,1.3,.053,-1.7,-1.4,-3.0,-0.3,38%,40%,20%,3%,NaN,-0.5,0.9,15,8,65,9,32,3,159,4,20
4,Santi Aldama,PF,22,MEM,77,20,1682,247,525,.470,94,266,.353,153,259,.591,.560,108,144,.750,85,286,371,97,45,48,60,143,696,13.9,.591,.507,.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,2.1,2.4,4.6,.130,-0.3,0.8,0.5,1.1,NaN,NaN,2%,69%,29%,2.2,-2.5,19,20,93,11,48,11,231,12,20


In [4]:
#here we have the data frame, which contains the average of all features
df_avg=avg_stats
df_avg.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,55,12,1140,6.2,12.8,...,.702,3.2,7.2,10.4,1.6,1.0,0.9,1.9,3.2,16.0
1,2,Steven Adams,C,29,MEM,42,42,1133,5.0,8.4,...,.364,6.8,8.6,15.4,3.1,1.1,1.5,2.5,3.1,11.5
2,3,Bam Adebayo,C,25,MIA,75,75,2598,8.3,15.4,...,.806,2.5,7.0,9.5,3.3,1.2,0.8,2.6,2.9,21.2
3,4,Ochai Agbaji,SG,22,UTA,59,22,1209,4.9,11.5,...,.812,1.3,2.3,3.6,2.0,0.5,0.4,1.2,2.9,13.9
4,5,Santi Aldama,PF,22,MEM,77,20,1682,5.3,11.2,...,.750,1.8,6.1,7.9,2.1,1.0,1.0,1.3,3.1,14.9


## Data Cleaning - df_final

In [10]:
print(df_final.info())
(df_final.isnull().sum()/len(df_final))*100

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18255 entries, 0 to 18254
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Player    18255 non-null  object
 1   Pos       18255 non-null  object
 2   Age       18255 non-null  object
 3   Tm        18255 non-null  object
 4   G         18255 non-null  object
 5   GS        18255 non-null  object
 6   MP        18255 non-null  object
 7   FG        18255 non-null  object
 8   FGA       18255 non-null  object
 9   FG%       18252 non-null  object
 10  3P        18255 non-null  object
 11  3PA       18255 non-null  object
 12  3P%       18231 non-null  object
 13  2P        18255 non-null  object
 14  2PA       18255 non-null  object
 15  2P%       18248 non-null  object
 16  eFG%      18252 non-null  object
 17  FT        18255 non-null  object
 18  FTA       18255 non-null  object
 19  FT%       18218 non-null  object
 20  ORB       18255 non-null  object
 21  DRB       18

Player    0.0
Pos       0.0
Age       0.0
Tm        0.0
G         0.0
         ... 
Shoot     0.0
Off.      0.0
PGA       0.0
And1      0.0
Blkd      0.0
Length: 65, dtype: float64

In [11]:
print(df_avg_final.info())
(df_avg_final.isnull().sum()/len(df_avg_final))*100

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      705 non-null    object
 1   Player  705 non-null    object
 2   Pos     705 non-null    object
 3   Age     705 non-null    object
 4   Tm      705 non-null    object
 5   G       705 non-null    object
 6   GS      705 non-null    object
 7   MP      705 non-null    object
 8   FG      705 non-null    object
 9   FGA     705 non-null    object
 10  FG%     702 non-null    object
 11  3P      705 non-null    object
 12  3PA     705 non-null    object
 13  3P%     681 non-null    object
 14  2P      705 non-null    object
 15  2PA     705 non-null    object
 16  2P%     698 non-null    object
 17  FT      705 non-null    object
 18  FTA     705 non-null    object
 19  FT%     668 non-null    object
 20  ORB     705 non-null    object
 21  DRB     705 non-null    object
 22  TRB     705 non-null    ob

Rk        0.000000
Player    0.000000
Pos       0.000000
Age       0.000000
Tm        0.000000
G         0.000000
GS        0.000000
MP        0.000000
FG        0.000000
FGA       0.000000
FG%       0.425532
3P        0.000000
3PA       0.000000
3P%       3.404255
2P        0.000000
2PA       0.000000
2P%       0.992908
FT        0.000000
FTA       0.000000
FT%       5.248227
ORB       0.000000
DRB       0.000000
TRB       0.000000
AST       0.000000
STL       0.000000
BLK       0.000000
TOV       0.000000
PF        0.000000
PTS       0.000000
dtype: float64

In [12]:
na_col=df_final.columns[df_final.isna().any()].to_list()
len(na_col)

14

In [13]:
#here, we can see all the rows which contain NaN values
#as we can see, the NaN values occur because the players did not generate any values
#therefore, I will replace all NaN values with zero
df_final[(df_final[na_col[0]].isna()) | (df_final[na_col[1]].isna()) | (df_final[na_col[2]].isna()) | (
    df_final[na_col[3]].isna()) | (df_final[na_col[4]].isna()) | (df_final[na_col[5]].isna()) | (df_final[na_col[6]].isna())
        | (df_final[na_col[7]].isna()) | (df_final[na_col[8]].isna())]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
16,Ryan Arcidiacono,PG,28,TOT,20,4,172,9,37,.243,8,23,.348,1,14,.071,.351,0,0,NaN,0,15,15,23,5,0,7,17,26,2.7,.351,.622,.000,0.0,10.0,5.0,16.9,1.4,0.0,15.9,11.1,-0.2,0.1,-0.2,-0.043,-7.3,-1.5,-8.8,-0.3,85%,15%,NaN,NaN,NaN,-10.3,-8.0,4,3,10,0,0,7,53,0,2
17,Ryan Arcidiacono,PG,28,NYK,11,0,26,1,5,.200,1,3,.333,0,2,.000,.300,0,0,NaN,0,4,4,2,2,0,1,3,3,3.1,.300,.600,.000,0.0,17.0,8.4,9.3,3.8,0.0,16.7,9.9,0.0,0.0,0.0,-0.012,-8.9,3.8,-5.1,0.0,21%,79%,NaN,NaN,NaN,7.4,4.3,0,1,1,0,0,0,5,0,0
18,Ryan Arcidiacono,PG,28,POR,9,4,146,8,32,.250,7,20,.350,1,12,.083,.359,0,0,NaN,0,11,11,21,3,0,6,14,23,2.6,.359,.625,.000,0.0,8.7,4.3,18.3,1.0,0.0,15.8,11.3,-0.2,0.0,-0.1,-0.049,-7.0,-2.4,-9.4,-0.3,96%,4%,NaN,NaN,NaN,-13.5,-10.2,4,2,9,0,0,7,48,0,2
21,Udoka Azubuike,C,23,UTA,36,4,359,59,72,.819,0,0,NaN,59,72,.819,.819,7,20,.350,31,88,119,11,6,15,18,34,125,16.1,.774,.000,.278,9.5,26.1,17.9,4.3,0.8,3.5,18.2,11.5,0.6,0.4,1.0,.134,-1.2,0.5,-0.6,0.1,NaN,NaN,NaN,2%,99%,-11.2,-11.1,2,5,29,2,9,0,24,2,3
1412,Bismack Biyombo,C,30,PHO,61,14,874,119,206,.578,0,0,NaN,119,206,.578,.578,25,70,.357,89,171,260,56,18,88,51,114,263,14.4,.555,.000,.340,11.0,22.1,16.5,8.7,1.0,9.0,17.7,14.0,0.2,1.6,1.9,.102,-3.4,2.5,-0.9,0.2,NaN,NaN,NaN,1%,99%,3.8,2.5,10,12,68,20,33,4,126,12,11
1417,Leandro Bolmaro,SF,22,UTA,14,0,68,3,20,.150,0,4,.000,3,16,.188,.150,0,0,NaN,4,3,7,7,3,1,7,10,6,-5.0,.150,.200,.000,6.5,4.7,5.6,12.3,2.1,1.2,25.9,16.6,-0.4,0.0,-0.3,-0.231,-12.4,-2.3,-14.7,-0.2,NaN,NaN,91%,9%,NaN,-0.1,1.0,5,1,7,0,0,4,17,0,1
1424,Jamaree Bouyea,PG,23,WAS,1,0,6,0,1,.000,0,1,.000,0,0,NaN,.000,0,0,NaN,0,1,1,0,0,0,0,1,0,-6.9,.000,1.000,.000,0.0,17.9,9.3,0.0,0.0,0.0,0.0,7.2,0.0,0.0,0.0,-0.150,-16.1,-1.7,-17.8,0.0,100%,NaN,NaN,NaN,NaN,36.4,37.7,0,0,0,0,0,0,0,0,0
2115,Moses Brown,C,23,TOT,36,1,294,61,96,.635,0,0,NaN,61,96,.635,.635,33,72,.458,57,82,139,4,4,14,15,39,155,22.2,.607,.000,.750,22.0,30.9,26.5,2.1,0.7,4.2,10.5,21.2,0.7,0.4,1.1,.179,0.6,-1.2,-0.6,0.1,NaN,NaN,NaN,NaN,100%,3.8,3.5,2,3,21,5,29,1,9,8,11
2116,Moses Brown,C,23,LAC,34,1,288,61,96,.635,0,0,NaN,61,96,.635,.635,33,72,.458,57,82,139,4,3,14,14,38,155,22.7,.607,.000,.750,22.4,31.5,27.0,2.2,0.5,4.3,9.9,21.5,0.7,0.4,1.1,.185,0.9,-1.3,-0.4,0.1,NaN,NaN,NaN,NaN,100%,4.8,4.5,2,3,20,5,29,1,9,8,11
2117,Moses Brown,C,23,BRK,2,0,6,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN,0,0,0,0,1,0,1,1,0,-2.6,NaN,NaN,NaN,0.0,0.0,0.0,0.0,8.1,0.0,100.0,7.4,0.0,0.0,0.0,-0.129,-12.7,2.8,-9.9,0.0,NaN,NaN,NaN,NaN,100%,-43.3,-44.6,0,0,1,0,0,0,0,0,0


In [14]:
#we do not have any NaN values any longer, but we can see that the data type is not correct
df_final.fillna(0,inplace=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18255 entries, 0 to 18254
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Player    18255 non-null  object
 1   Pos       18255 non-null  object
 2   Age       18255 non-null  object
 3   Tm        18255 non-null  object
 4   G         18255 non-null  object
 5   GS        18255 non-null  object
 6   MP        18255 non-null  object
 7   FG        18255 non-null  object
 8   FGA       18255 non-null  object
 9   FG%       18255 non-null  object
 10  3P        18255 non-null  object
 11  3PA       18255 non-null  object
 12  3P%       18255 non-null  object
 13  2P        18255 non-null  object
 14  2PA       18255 non-null  object
 15  2P%       18255 non-null  object
 16  eFG%      18255 non-null  object
 17  FT        18255 non-null  object
 18  FTA       18255 non-null  object
 19  FT%       18255 non-null  object
 20  ORB       18255 non-null  object
 21  DRB       18

In [15]:
df_final=df_final[df_final['Age']!='Age']
df_final.head(25)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
0,Precious Achiuwa,C,23,TOR,55,12,1140,196,404,.485,29,108,.269,167,296,.564,.521,87,124,.702,100,228,328,50,31,30,59,102,508,15.2,.554,.267,.307,9.3,24.4,16.3,6.3,1.3,2.6,11.4,19.4,0.8,1.4,2.2,.093,-1.4,-0.8,-2.3,-0.1,0,0,0,17%,83%,-1.5,-3.8,8,21,53,19,58,7,128,15,31
1,Steven Adams,C,29,MEM,42,42,1133,157,263,.597,0,1,.000,157,262,.599,.597,47,129,.364,214,271,485,97,36,46,79,98,361,17.5,.564,.004,.490,20.1,25.3,22.7,11.2,1.5,3.7,19.8,14.6,1.3,2.1,3.4,.144,-0.3,0.9,0.6,0.7,0,0,0,0,100%,9.5,8.3,40,10,40,23,51,7,223,15,14
2,Bam Adebayo,C,25,MIA,75,75,2598,602,1114,.540,1,12,.083,601,1102,.545,.541,324,402,.806,184,504,688,240,88,61,187,208,1529,20.1,.592,.011,.361,8.0,23.6,15.5,15.9,1.7,2.4,12.7,25.2,3.6,3.8,7.4,.137,0.8,0.8,1.5,2.3,0,0,0,0,100%,1.7,5.4,60,46,83,50,194,10,587,37,49
3,Ochai Agbaji,SG,22,UTA,59,22,1209,165,386,.427,81,228,.355,84,158,.532,.532,56,69,.812,43,78,121,67,16,15,41,99,467,9.5,.561,.591,.179,3.9,6.9,5.4,7.5,0.6,1.0,9.0,15.8,0.9,0.4,1.3,.053,-1.7,-1.4,-3.0,-0.3,38%,40%,20%,3%,0,-0.5,0.9,15,8,65,9,32,3,159,4,20
4,Santi Aldama,PF,22,MEM,77,20,1682,247,525,.470,94,266,.353,153,259,.591,.560,108,144,.750,85,286,371,97,45,48,60,143,696,13.9,.591,.507,.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,2.1,2.4,4.6,.130,-0.3,0.8,0.5,1.1,0,0,2%,69%,29%,2.2,-2.5,19,20,93,11,48,11,231,12,20
5,Nickeil Alexander-Walker,SG,24,TOT,59,3,884,131,295,.444,61,159,.384,70,136,.515,.547,40,60,.667,15,86,101,108,32,21,55,88,363,11.6,.565,.539,.203,1.9,10.5,6.3,16.7,1.7,2.0,14.6,17.9,0.3,0.8,1.1,.062,-1.4,0.4,-0.9,0.2,2%,39%,51%,8%,0,-2.5,-2.0,38,9,59,3,25,23,262,6,13
6,Nickeil Alexander-Walker,SG,24,UTA,36,3,528,83,170,.488,35,87,.402,48,83,.578,.591,27,39,.692,8,51,59,76,24,14,45,58,228,13.0,.609,.512,.229,1.7,10.3,6.0,19.9,2.2,2.2,19.4,18.4,0.3,0.5,0.8,.074,-0.6,1.1,0.5,0.3,0,34%,56%,10%,0,-2.3,-1.4,31,8,38,3,16,16,183,3,5
7,Nickeil Alexander-Walker,SG,24,MIN,23,0,356,48,125,.384,26,72,.361,22,53,.415,.488,13,21,.619,7,35,42,32,8,7,10,30,135,9.6,.503,.576,.168,2.2,10.8,6.6,12.0,1.1,1.7,6.9,17.3,0.0,0.3,0.3,.044,-2.5,-0.5,-3.0,-0.1,5%,48%,42%,5%,0,-2.9,-3.0,7,1,21,0,9,7,79,3,8
8,Grayson Allen,SG,27,MIL,72,70,1972,245,557,.440,146,366,.399,99,191,.518,.571,114,126,.905,61,176,237,163,62,14,72,117,750,12.3,.612,.657,.226,3.4,9.0,6.3,10.9,1.5,0.6,10.5,14.6,2.8,2.2,5.1,.123,-0.6,0.7,0.1,1.0,5%,75%,21%,0,0,5.7,4.1,49,11,68,1,44,23,409,7,22
9,Jarrett Allen,C,24,CLE,68,68,2220,403,626,.644,1,10,.100,402,616,.653,.645,162,221,.733,221,445,666,113,54,84,93,153,969,19.9,.670,.016,.353,11.7,23.6,17.7,7.5,1.2,3.5,11.4,16.4,5.5,4.0,9.5,.205,1.3,1.0,2.4,2.4,0,0,0,0,100%,7.9,4.9,41,18,90,17,100,13,271,20,43


In [17]:
# first transformation approach for non period seperated string values
cols=['Age','G', 'GS', 'MP', 'FG', 'FGA','3P','3PA','2P', '2PA','FT', 'FTA','ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

for x in cols:
    df_final[x]=pd.to_numeric(df_final[x])
df_final.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
0,Precious Achiuwa,C,23,TOR,55,12,1140,196,404,.485,29,108,.269,167,296,.564,.521,87,124,.702,100,228,328,50,31,30,59,102,508,15.2,.554,.267,.307,9.3,24.4,16.3,6.3,1.3,2.6,11.4,19.4,0.8,1.4,2.2,0.093,-1.4,-0.8,-2.3,-0.1,0,0,0,17%,83%,-1.5,-3.8,8,21,53,19,58,7,128,15,31
1,Steven Adams,C,29,MEM,42,42,1133,157,263,.597,0,1,.000,157,262,.599,.597,47,129,.364,214,271,485,97,36,46,79,98,361,17.5,.564,.004,.490,20.1,25.3,22.7,11.2,1.5,3.7,19.8,14.6,1.3,2.1,3.4,0.144,-0.3,0.9,0.6,0.7,0,0,0,0,100%,9.5,8.3,40,10,40,23,51,7,223,15,14
2,Bam Adebayo,C,25,MIA,75,75,2598,602,1114,.540,1,12,.083,601,1102,.545,.541,324,402,.806,184,504,688,240,88,61,187,208,1529,20.1,.592,.011,.361,8.0,23.6,15.5,15.9,1.7,2.4,12.7,25.2,3.6,3.8,7.4,0.137,0.8,0.8,1.5,2.3,0,0,0,0,100%,1.7,5.4,60,46,83,50,194,10,587,37,49
3,Ochai Agbaji,SG,22,UTA,59,22,1209,165,386,.427,81,228,.355,84,158,.532,.532,56,69,.812,43,78,121,67,16,15,41,99,467,9.5,.561,.591,.179,3.9,6.9,5.4,7.5,0.6,1.0,9.0,15.8,0.9,0.4,1.3,0.053,-1.7,-1.4,-3.0,-0.3,38%,40%,20%,3%,0,-0.5,0.9,15,8,65,9,32,3,159,4,20
4,Santi Aldama,PF,22,MEM,77,20,1682,247,525,.470,94,266,.353,153,259,.591,.560,108,144,.750,85,286,371,97,45,48,60,143,696,13.9,.591,.507,.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,2.1,2.4,4.6,0.130,-0.3,0.8,0.5,1.1,0,0,2%,69%,29%,2.2,-2.5,19,20,93,11,48,11,231,12,20


In [18]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 18254
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    679 non-null    object 
 1   Pos       679 non-null    object 
 2   Age       679 non-null    int64  
 3   Tm        679 non-null    object 
 4   G         679 non-null    int64  
 5   GS        679 non-null    int64  
 6   MP        679 non-null    int64  
 7   FG        679 non-null    int64  
 8   FGA       679 non-null    int64  
 9   FG%       679 non-null    object 
 10  3P        679 non-null    int64  
 11  3PA       679 non-null    int64  
 12  3P%       679 non-null    object 
 13  2P        679 non-null    int64  
 14  2PA       679 non-null    int64  
 15  2P%       679 non-null    object 
 16  eFG%      679 non-null    object 
 17  FT        679 non-null    int64  
 18  FTA       679 non-null    int64  
 19  FT%       679 non-null    object 
 20  ORB       679 non-null    int6

In [19]:
#second transformation approach for period seperated values
cols_per=['FG%','3P%','2P%','eFG%','TS%','3PAr','FTr','FT%']
for x in cols_per:
    df_final[x]=pd.to_numeric(df_final[x].replace({0:'0.0','1.000':'1.100'}).apply(lambda x: x.split(".")[1]))/1000

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 18254
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    679 non-null    object 
 1   Pos       679 non-null    object 
 2   Age       679 non-null    int64  
 3   Tm        679 non-null    object 
 4   G         679 non-null    int64  
 5   GS        679 non-null    int64  
 6   MP        679 non-null    int64  
 7   FG        679 non-null    int64  
 8   FGA       679 non-null    int64  
 9   FG%       679 non-null    float64
 10  3P        679 non-null    int64  
 11  3PA       679 non-null    int64  
 12  3P%       679 non-null    float64
 13  2P        679 non-null    int64  
 14  2PA       679 non-null    int64  
 15  2P%       679 non-null    float64
 16  eFG%      679 non-null    float64
 17  FT        679 non-null    int64  
 18  FTA       679 non-null    int64  
 19  FT%       679 non-null    float64
 20  ORB       679 non-null    int6

In [21]:
cols_pos=['PG%','SG%','SF%','PF%','C%']
for col_pos in cols_pos:
    df_final[col_pos]=pd.to_numeric(df_final[col_pos].replace(0,'0%').apply(lambda x: x.split("%")[0]))/100

In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 18254
Data columns (total 65 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    679 non-null    object 
 1   Pos       679 non-null    object 
 2   Age       679 non-null    int64  
 3   Tm        679 non-null    object 
 4   G         679 non-null    int64  
 5   GS        679 non-null    int64  
 6   MP        679 non-null    int64  
 7   FG        679 non-null    int64  
 8   FGA       679 non-null    int64  
 9   FG%       679 non-null    float64
 10  3P        679 non-null    int64  
 11  3PA       679 non-null    int64  
 12  3P%       679 non-null    float64
 13  2P        679 non-null    int64  
 14  2PA       679 non-null    int64  
 15  2P%       679 non-null    float64
 16  eFG%      679 non-null    float64
 17  FT        679 non-null    int64  
 18  FTA       679 non-null    int64  
 19  FT%       679 non-null    float64
 20  ORB       679 non-null    int6

In [23]:
df_final['Shooting_Fouls_Committed']=pd.to_numeric(df_final.iloc[:,58])
df_final['Offensive_Fouls_Committed']=pd.to_numeric(df_final.iloc[:,59])
df_final['Shooting_Fouls_Drawn']=pd.to_numeric(df_final.iloc[:,60])
df_final['Offensive_Fouls_Drawn']=pd.to_numeric(df_final.iloc[:,61])
df_final.drop(['Shoot','Off.'], axis=1,inplace=True)

In [24]:
cols_adv=['OnCourt','On-Off','BadPass','LostBall','PGA','And1','Blkd']
for adv in cols_adv:
    df_final[adv]=pd.to_numeric(df_final[adv])

In [25]:
df_final=df_final.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
                          34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,61,62,63,64,58,59,60]]
df_final.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shooting_Fouls_Committed,Offensive_Fouls_Committed,Shooting_Fouls_Drawn,Offensive_Fouls_Drawn,PGA,And1,Blkd
0,Precious Achiuwa,C,23,TOR,55,12,1140,196,404,0.485,29,108,0.269,167,296,0.564,0.521,87,124,0.702,100,228,328,50,31,30,59,102,508,15.2,0.554,0.267,0.307,9.3,24.4,16.3,6.3,1.3,2.6,11.4,19.4,0.8,1.4,2.2,0.093,-1.4,-0.8,-2.3,-0.1,0.00,0.0,0.00,0.17,0.83,-1.5,-3.8,8,21,53,19,58,7,128,15,31
1,Steven Adams,C,29,MEM,42,42,1133,157,263,0.597,0,1,0.000,157,262,0.599,0.597,47,129,0.364,214,271,485,97,36,46,79,98,361,17.5,0.564,0.004,0.490,20.1,25.3,22.7,11.2,1.5,3.7,19.8,14.6,1.3,2.1,3.4,0.144,-0.3,0.9,0.6,0.7,0.00,0.0,0.00,0.00,1.00,9.5,8.3,40,10,40,23,51,7,223,15,14
2,Bam Adebayo,C,25,MIA,75,75,2598,602,1114,0.540,1,12,0.083,601,1102,0.545,0.541,324,402,0.806,184,504,688,240,88,61,187,208,1529,20.1,0.592,0.011,0.361,8.0,23.6,15.5,15.9,1.7,2.4,12.7,25.2,3.6,3.8,7.4,0.137,0.8,0.8,1.5,2.3,0.00,0.0,0.00,0.00,1.00,1.7,5.4,60,46,83,50,194,10,587,37,49
3,Ochai Agbaji,SG,22,UTA,59,22,1209,165,386,0.427,81,228,0.355,84,158,0.532,0.532,56,69,0.812,43,78,121,67,16,15,41,99,467,9.5,0.561,0.591,0.179,3.9,6.9,5.4,7.5,0.6,1.0,9.0,15.8,0.9,0.4,1.3,0.053,-1.7,-1.4,-3.0,-0.3,0.38,0.4,0.20,0.03,0.00,-0.5,0.9,15,8,65,9,32,3,159,4,20
4,Santi Aldama,PF,22,MEM,77,20,1682,247,525,0.470,94,266,0.353,153,259,0.591,0.560,108,144,0.750,85,286,371,97,45,48,60,143,696,13.9,0.591,0.507,0.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,2.1,2.4,4.6,0.130,-0.3,0.8,0.5,1.1,0.00,0.0,0.02,0.69,0.29,2.2,-2.5,19,20,93,11,48,11,231,12,20


These were all the steps required to set up the data frame of one NBA season. Next, I will create a def function, allowing us to do all the transformation steps easily in one function.

In [2]:
#def function
def NBA_tranform(x,z,w,y):
    pd.set_option('display.max_columns',100)
    total_stats=pd.read_html(x)[0]
    time.sleep(5)
    advanced=pd.read_html(z)[0]
    advanced.drop(['Unnamed: 19','Unnamed: 24'], axis=1, inplace=True)
    time.sleep(5)
    play_by_play=pd.read_html(w)[0]
    play_by_play=play_by_play.droplevel(0,axis=1)
    df_final=pd.merge(total_stats,advanced, how='left', on=['Player','Pos','Age','Tm','G'])
    df_final.drop(['Rk_x','Rk_y','MP_y'], axis=1, inplace=True)
    df_final.rename(columns={'MP_x':'MP'},inplace=True)
    df_final=pd.merge(df_final, play_by_play, how='left', on=['Player','Pos','Age','Tm','G'])
    df_final.drop(['Rk','MP_y'], axis=1, inplace=True)
    df_final.rename(columns={'MP_x':'MP'},inplace=True)
    df_final.fillna(0,inplace=True)
    df_final=df_final[df_final['Age']!='Age']
    
    cols=['Age','G', 'GS', 'MP', 'FG', 'FGA','3P','3PA','2P', '2PA','FT', 'FTA','ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER','ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
    for col in cols:
        df_final[col]=pd.to_numeric(df_final[col])
    
    cols_per=['FG%','3P%','2P%','eFG%','TS%','3PAr','FTr','FT%']
    for per in cols_per:
        df_final[per]=pd.to_numeric(df_final[per].replace({0:'0.0','1.000':'1.100'}).apply(lambda x: x.split(".")[1]))/1000
        
    cols_pos=['PG%','SG%','SF%','PF%','C%']
    for col_pos in cols_pos:
        df_final[col_pos]=pd.to_numeric(df_final[col_pos].replace(0,'0%').apply(lambda x: x.split("%")[0]))/100
        
    df_final['Shooting_Fouls_Committed']=pd.to_numeric(df_final.iloc[:,58])
    df_final['Offensive_Fouls_Committed']=pd.to_numeric(df_final.iloc[:,59])
    df_final['Shooting_Fouls_Drawn']=pd.to_numeric(df_final.iloc[:,60])
    df_final['Offensive_Fouls_Drawn']=pd.to_numeric(df_final.iloc[:,61])
    df_final.drop(['Shoot','Off.'], axis=1,inplace=True)
    
    cols_adv=['OnCourt','On-Off','BadPass','LostBall','PGA','And1','Blkd']
    for adv in cols_adv:
        df_final[adv]=pd.to_numeric(df_final[adv])
    
    df_final=df_final.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
                          34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,61,62,63,64,58,59,60]]
    
    df_final['Year']=y
    return df_final

In [3]:
#we use a dictionary to save the different data frames
dfs={}
for x in range(1997,2024):
    dfs["df_"+str(x)]=NBA_tranform('https://www.basketball-reference.com/leagues/NBA_{}_totals.html'.format(x),
                      'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(x),
                        'https://www.basketball-reference.com/leagues/NBA_{}_play-by-play.html'.format(x),x)
    locals().update(dfs)

In [4]:
list(dfs.keys())[2:]

['df_1999',
 'df_2000',
 'df_2001',
 'df_2002',
 'df_2003',
 'df_2004',
 'df_2005',
 'df_2006',
 'df_2007',
 'df_2008',
 'df_2009',
 'df_2010',
 'df_2011',
 'df_2012',
 'df_2013',
 'df_2014',
 'df_2015',
 'df_2016',
 'df_2017',
 'df_2018',
 'df_2019',
 'df_2020',
 'df_2021',
 'df_2022',
 'df_2023']

In [5]:
#here, we merge all the different data frames
df_nba=list(dfs.keys())[2:]

df=pd.concat([pd.DataFrame(dfs['df_1997']),pd.DataFrame(dfs['df_1998'])])
for x in df_nba:
    df=pd.concat([df,pd.DataFrame(dfs[x])])

In [6]:
#as we can see, we successfully merge all tables 
df['Year'].unique()

array([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023], dtype=int64)

In [8]:
df.to_csv('NBA_1997_2023.csv')

## Data Cleaning - df_avg

In [41]:
#again, we face the same issue as we had with the total stats, NaN values can be replaced with 0.0
df_avg[df_avg['2P%'].isna()]

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
74,53,Jamaree Bouyea,PG,23,WAS,1,0,6,0.0,6.0,.000,0.0,6.0,.000,0.0,0.0,NaN,0.0,0.0,NaN,0.0,6.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0
92,66,Moses Brown,C,23,BRK,2,0,6,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,6.0,0.0,6.0,6.0,0.0
203,151,Michael Foster Jr.,PF,20,PHI,1,0,1,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219,166,Jacob Gilyard,PG,24,MEM,1,0,41,0.9,2.6,.333,0.9,2.6,.333,0.0,0.0,NaN,0.0,0.0,NaN,0.0,3.5,3.5,6.1,2.6,0.0,1.8,2.6,2.6
349,263,Trevor Keels,SG,19,NYK,3,0,8,4.5,18.0,.250,4.5,18.0,.250,0.0,0.0,NaN,0.0,0.0,NaN,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,13.5
637,482,Stanley Umude,SG,23,DET,1,0,2,0.0,18.0,.000,0.0,18.0,.000,0.0,0.0,NaN,36.0,36.0,1.000,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0,36.0
677,515,Alondes Williams,SG,23,BRK,1,0,5,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,7.2,7.2,0.0,0.0,0.0,14.4,7.2,0.0


In [5]:
#next, I will check if we have the same issue where table headers appear as rows
print(df_avg['Age'].unique())
#as we can see, we face the same issue, and we fix it the same way
df_avg=df_avg[df_avg['Age']!='Age']
df_avg.fillna('0.0',inplace=True)

['23' '29' '25' '22' '24' '27' '28' '30' '20' 'Age' '21' '19' '32' '34'
 '26' '33' '31' '35' '36' '37' '42' '39' '38']


C:\Users\gandj\AppData\Local\Temp\ipykernel_43944\1670489501.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg.fillna('0.0',inplace=True)


Next, I have to change the data type of the different features; this time, I will not do every step in detail; I will provide the solution immediately.

In [6]:
df_avg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 704
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      679 non-null    object
 1   Player  679 non-null    object
 2   Pos     679 non-null    object
 3   Age     679 non-null    object
 4   Tm      679 non-null    object
 5   G       679 non-null    object
 6   GS      679 non-null    object
 7   MP      679 non-null    object
 8   FG      679 non-null    object
 9   FGA     679 non-null    object
 10  FG%     679 non-null    object
 11  3P      679 non-null    object
 12  3PA     679 non-null    object
 13  3P%     679 non-null    object
 14  2P      679 non-null    object
 15  2PA     679 non-null    object
 16  2P%     679 non-null    object
 17  FT      679 non-null    object
 18  FTA     679 non-null    object
 19  FT%     679 non-null    object
 20  ORB     679 non-null    object
 21  DRB     679 non-null    object
 22  TRB     679 non-null    ob

In [7]:
#transforming values from string to numeric
avg_cols=df_avg.drop(['Rk','Player','Pos','Tm'],axis=1).columns.to_list()
for x in avg_cols:
    df_avg[x]=pd.to_numeric(df_avg[x])

C:\Users\gandj\AppData\Local\Temp\ipykernel_43944\3500813720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg[x]=pd.to_numeric(df_avg[x])


In [9]:
df_avg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679 entries, 0 to 704
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      679 non-null    object 
 1   Player  679 non-null    object 
 2   Pos     679 non-null    object 
 3   Age     679 non-null    int64  
 4   Tm      679 non-null    object 
 5   G       679 non-null    int64  
 6   GS      679 non-null    int64  
 7   MP      679 non-null    int64  
 8   FG      679 non-null    float64
 9   FGA     679 non-null    float64
 10  FG%     679 non-null    float64
 11  3P      679 non-null    float64
 12  3PA     679 non-null    float64
 13  3P%     679 non-null    float64
 14  2P      679 non-null    float64
 15  2PA     679 non-null    float64
 16  2P%     679 non-null    float64
 17  FT      679 non-null    float64
 18  FTA     679 non-null    float64
 19  FT%     679 non-null    float64
 20  ORB     679 non-null    float64
 21  DRB     679 non-null    float64
 22  TR

In [2]:
def NBA_Avg_transform(x,y):
    pd.set_option('display.max_columns',100)
    df_avg=pd.read_html(x)[0]
    df_avg=df_avg[df_avg['Age']!='Age']
    df_avg.fillna('0.0',inplace=True)
    avg_cols=df_avg.drop(['Rk','Player','Pos','Tm'],axis=1).columns.to_list()
    for x in avg_cols:
        df_avg[x]=pd.to_numeric(df_avg[x])
    df_avg['Year']=y
    df_avg.drop(['Rk'],axis=1,inplace=True)
    return df_avg

In [3]:
#we use a dictionary to to save the different data frames
dfsavg={}
for x in range(1984,2024):
    dfsavg["df_avg_"+str(x)]=NBA_Avg_transform('https://www.basketball-reference.com/leagues/NBA_{}_per_minute.html'.format(x),x)
    locals().update(dfsavg)
    time.sleep(10)

df_avg_nba=list(dfsavg.keys())[2:]
df_avg=pd.concat([pd.DataFrame(dfsavg['df_avg_1984']),pd.DataFrame(dfsavg['df_avg_1985'])])
for x in df_avg_nba:
    df_avg=pd.concat([df_avg,pd.DataFrame(dfsavg[x])])

In [4]:
df_avg.to_csv('NBA_Avg_Stats.csv')

In [5]:
df_avg.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Kareem Abdul-Jabbar*,C,36,LAL,80,80,2622,9.8,17.0,0.578,0.0,0.0,0.000,9.8,17.0,0.579,3.9,5.4,0.723,2.3,5.7,8.1,2.9,0.8,2.0,3.0,2.9,23.6,1984
1,Alvan Adams,C,29,PHO,70,13,1452,6.7,14.4,0.462,0.0,0.1,0.000,6.7,14.3,0.465,3.3,4.0,0.825,2.9,5.0,7.9,5.4,1.8,0.8,2.9,4.8,16.6,1984
2,Mark Aguirre,SF,24,DAL,79,79,2900,11.5,21.9,0.524,0.2,0.7,0.268,11.3,21.2,0.532,5.8,7.7,0.749,2.0,3.8,5.8,4.4,1.0,0.3,3.5,3.1,28.9,1984
3,Danny Ainge,SG,24,BOS,71,3,1154,5.2,11.3,0.460,0.2,0.7,0.273,5.0,10.6,0.472,1.4,1.7,0.821,0.9,2.7,3.6,5.1,1.3,0.1,2.2,4.5,12.0,1984
4,J.J. Anderson,SF,23,UTA,48,0,311,6.4,15.0,0.423,0.0,0.3,0.000,6.4,14.7,0.433,1.4,3.4,0.414,4.4,2.9,7.3,2.5,1.7,1.0,2.3,3.2,14.1,1984


In [6]:
df_avg['Year'].unique()

array([1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int64)

## MVPs over the different Decades

In [21]:
url="https://www.basketball-reference.com/leagues/NBA_2023_totals.html"
data=requests.get(url).text
soup=BeautifulSoup(data,'lxml')
test=soup.find('div', id ='info')
output=[]
for i in test.find_all('p'):
    loop=i.text
    output.append(loop)
output=output[2:]
output

['League Champion: Denver Nuggets',
 'Most Valuable Player: Joel Embiid (33.1/10.2/4.2)',
 'Rookie of the Year: Paolo Banchero (20.0/6.9/3.7)',
 'PPG Leader: Joel Embiid (33.1)',
 'RPG Leader: Domantas Sabonis (12.3)',
 'APG Leader: James Harden (10.7)',
 'WS Leader: Nikola Jokić (14.9)']

In [43]:
dic_title={}
for x in output:
    k,v=x.split(':')
    dic_title[k]=v.strip()
dic_title

{'League Champion': 'Denver Nuggets',
 'Most Valuable Player': 'Joel Embiid (33.1/10.2/4.2)',
 'Rookie of the Year': 'Paolo Banchero (20.0/6.9/3.7)',
 'PPG Leader': 'Joel Embiid (33.1)',
 'RPG Leader': 'Domantas Sabonis (12.3)',
 'APG Leader': 'James Harden (10.7)',
 'WS Leader': 'Nikola Jokić (14.9)'}

In [55]:
df_season=pd.DataFrame(dic_title,index=range(0,1))
df_season['Most Valuable Player']=df_season['Most Valuable Player'].apply(lambda x:x.split('(')[0].strip())
df_season['Rookie of the Year']=df_season['Rookie of the Year'].apply(lambda x:x.split('(')[0].strip())
df_season['PPG Leader']=df_season['PPG Leader'].apply(lambda x:x.split('(')[0].strip())
df_season['RPG Leader']=df_season['RPG Leader'].apply(lambda x:x.split('(')[0].strip())
df_season['APG Leader']=df_season['APG Leader'].apply(lambda x:x.split('(')[0].strip())
df_season['WS Leader']=df_season['WS Leader'].apply(lambda x:x.split('(')[0].strip())
df_season['Year']=2023
df_season

,League Champion,Most Valuable Player,Rookie of the Year,PPG Leader,RPG Leader,APG Leader,WS Leader,Year
0,Denver Nuggets,Joel Embiid,Paolo Banchero,Joel Embiid,Domantas Sabonis,James Harden,Nikola Jokić,2023


In [77]:
def web_seasonal_outcome(url,year):
    #creating the web-scrapper
    url=url
    data=requests.get(url).text
    soup=BeautifulSoup(data,'lxml')
    test=soup.find('div', id ='info')
    output=[]
    for i in test.find_all('p'):
        loop=i.text
        output.append(loop)
    output=output[2:]
    output
    
    #creating with the web-scrapper output a dictionary
    dic_title={}
    for x in output:
        k,v=x.split(':')
        dic_title[k]=v.strip()
    dic_title
    
    #creating with the dictionary a data frame
    df_season=pd.DataFrame(dic_title,index=range(0,1))
    cols=list(df_season.columns[1:])
    for col in cols:
       df_season[col]=df_season[col].apply(lambda x:x.split('(')[0].strip())
    df_season['Year']=year
    return df_season

In [94]:
#we use a dictionary to to save the different data frames
df_season_dict={}
for x in range(1997,2024):
    df_season_dict["df_season_"+str(x)]=web_seasonal_outcome(
        'https://www.basketball-reference.com/leagues/NBA_{}_per_minute.html'.format(x),x)
    locals().update(df_season_dict)
    time.sleep(10)

#here, we merge all the different data frames
df_season_key=list(df_season_dict.keys())[2:]
df_season=pd.concat([pd.DataFrame(df_season_dict['df_season_1997']),pd.DataFrame(df_season_dict['df_season_1998'])])
for x in df_season_key:
    df_season=pd.concat([df_season,pd.DataFrame(df_season_dict[x])])

In [95]:
df_season.rename(columns={'WS Leader':'Win Share Leader'}, inplace=True)
df_season=df_season.reset_index().drop(['index'],axis=1)
df_season.to_csv('Season_output_1997_2023.csv')

In [107]:
df_season_avg_dict={}
for x in range(1984,2024):
    df_season_avg_dict["df_season_"+str(x)]=web_seasonal_outcome(
        'https://www.basketball-reference.com/leagues/NBA_{}_per_minute.html'.format(x),x)
    locals().update(df_season_avg_dict)
    time.sleep(10)
    
df_season_avg_key=list(df_season_avg_dict.keys())[2:]
df_avg_season=pd.concat([pd.DataFrame(df_season_avg_dict['df_season_1984']),pd.DataFrame(df_season_avg_dict['df_season_1985'])])
for x in df_season_avg_key:
    df_avg_season=pd.concat([df_avg_season,pd.DataFrame(df_season_avg_dict[x])])

In [108]:
df_avg_season

,League Champion,Most Valuable Player,Rookie of the Year,PPG Leader,RPG Leader,APG Leader,WS Leader,Year
0,Boston Celtics,Larry Bird,Ralph Sampson,Adrian Dantley,Moses Malone,Magic Johnson,Adrian Dantley,1984
0,Los Angeles Lakers,Larry Bird,Michael Jordan,Bernard King,Moses Malone,Isiah Thomas,Larry Bird,1985
0,Boston Celtics,Larry Bird,Patrick Ewing,Dominique Wilkins,Bill Laimbeer,Magic Johnson,Larry Bird,1986
0,Los Angeles Lakers,Magic Johnson,Chuck Person,Michael Jordan,Charles Barkley,Magic Johnson,Michael Jordan,1987
0,Los Angeles Lakers,Michael Jordan,Mark Jackson,Michael Jordan,Michael Cage,John Stockton,Michael Jordan,1988
0,Detroit Pistons,Magic Johnson,Mitch Richmond,Michael Jordan,Hakeem Olajuwon,John Stockton,Michael Jordan,1989
0,Detroit Pistons,Magic Johnson,David Robinson,Michael Jordan,Hakeem Olajuwon,John Stockton,Michael Jordan,1990
0,Chicago Bulls,Michael Jordan,Derrick Coleman,Michael Jordan,David Robinson,John Stockton,Michael Jordan,1991
0,Chicago Bulls,Michael Jordan,Larry Johnson,Michael Jordan,Dennis Rodman,John Stockton,Michael Jordan,1992
0,Chicago Bulls,Charles Barkley,Shaquille O'Neal,Michael Jordan,Dennis Rodman,John Stockton,Michael Jordan,1993


In [109]:
df_avg_season.rename(columns={'WS Leader':'Win Share Leader'}, inplace=True)
df_avg_season=df_avg_season.reset_index().drop(['index'],axis=1)
df_avg_season.to_csv('Season_avg_output_1984_2023.csv')

# Source

Tiodorovic, K. (2020). Efficiency- The Most Important Stat in Basketball. [online] BenchBoss. Available at: https://benchboss.ai/efficiency-the-most-important-stat-in-basketball/

STACK NJ/NY (2021). Redirect Notice. [online] Available at: https://www.google.com/url?q=https://www.stacknj.com/2021/11/07/most-important-statistics-in-winning-basketball-games/&source=gmail&ust=1704586413019000&usg=AOvVaw1JAxWYZBJwFp10YUNnuIDS

basketball stats, (2021). 5 Advanced Basketball Stats Every Team Should Track. [online] Available at: https://www.basketballforcoaches.com/basketball-stats/